# Домашнее задание 2.

### Гриневская Катя, БКЛ182

#### Загрузка данных


Импорты

In [4]:
import pandas as pd
import gzip
import json
from tqdm.auto import tqdm
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("english")
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import spacy
nlp = spacy.load("en_core_web_sm")
from collections import Counter
import gensim
from string import punctuation

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Я выбрала отзывы о товарых для животных, причём у каждого товара/пользователя есть минимум 5 отзывов.

In [ ]:
! wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Pet_Supplies_5.json.gz

--2021-12-04 11:24:09--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Pet_Supplies_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306131006 (292M) [application/octet-stream]
Saving to: ‘Pet_Supplies_5.json.gz.1’

Pet_Supplies_5.json 100%[===================>] 291.95M  20.7MB/s    in 15s     

2021-12-04 11:24:24 (19.8 MB/s) - ‘Pet_Supplies_5.json.gz.1’ saved [306131006/306131006]



In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Pet_Supplies_5.json.gz')

In [ ]:
df[550:570]

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
550,5.0,NaN,True,"06 17, 2016",A23A3FWNF2XFV0,7310172001,Amazon Customer,dogs love it,Five Stars,1466121600,NaN,NaN
551,5.0,NaN,True,"05 28, 2016",AAQF1FIR68QBY,7310172001,Carol A. P.,My dog goes crazy for these! He absolutely lo...,My dog begs for these!!,1464393600,NaN,NaN
552,5.0,2,True,"03 7, 2016",AGDP18EW9AE37,7310172001,D. Owen,The dogs say thanks for the quick delivery. I ...,Dogs love the liver.,1457308800,NaN,NaN
553,4.0,NaN,True,"02 28, 2016",AVEND03HPIQT1,7310172001,Lenna,This tub was ridiculously difficult to open - ...,"Pup seems to like, but we're not thrilled",1456617600,NaN,NaN
554,5.0,NaN,True,"01 4, 2016",A1PMBAR3B8JTTY,7310172001,Moms Amazon,My dog loves these,Five Stars,1451865600,NaN,NaN
555,4.0,NaN,True,"09 23, 2015",A1AC8DR6J95DO0,7310172001,Kindle Customer,My dog (maltese) loves them. I do not buy the...,a little too pricey,1442966400,NaN,NaN
556,4.0,NaN,True,"06 14, 2015",A3A968FR6DBV1P,7310172001,Douglas Lambert,Foxy the Pomeranian says this is one of her fa...,Great treat for your pet,1434240000,NaN,NaN
557,4.0,NaN,True,"03 18, 2015",A1QPXODP6YNNF8,8029311600,Colorado Girl,This seems to be working on both my dogs teeth...,Seems to be doing the job,1426636800,NaN,NaN
558,2.0,NaN,False,"06 5, 2014",A1EKPNO3QI2Y3F,8029311600,Georgie,and has quit a lot of plaque on her teeth. I ...,My dog has been on this for many months..........,1401926400,NaN,NaN
559,5.0,NaN,True,"01 8, 2013",A2UUXCSS3JLL6C,8029311600,MARIA FATA,We mix this with our dog's drinking water. It...,Great way to keep teeth healthy!,1357603200,NaN,NaN


In [ ]:
df['reviewText'][566]

'Thank You!!!  Everything was great....exactly what I ordered.'

In [ ]:
len(df)

2098325

### Задание 1.
Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?



**1.** Предложенный вариант с bootstrapping-ом, кажется, будет работать плохо. Пользователи редко используют названия товаров в своих отзывов, чаще заменяют название товара местоимениями (*it, this, these*), назвают товар словосочетаниями вроде *this tub, these perches* или же не называют товар совсем, и отзыв представляет из себя что-то похожее на *Thank You!!!  Everything was great....exactly what I ordered.*

Итак, **достоинства**: лёгкая реализация, никаких дополнительных данных.

**Недостатки**: почти ничего полезного не найдём.

**2.** Можно взять из Metadata названия, и id товаров. Затем сопоставить название товара с отзывом и искать упоминания товара из названия в отзыве. К названию товара можно с помощью WordNet или векторной моели подобрать синонимы и/или гиперонимы и их тоже искать в отзыве.

**Дополнительные данные**: Metadata, WordNet и/или векторная модель

**Достоинства**: понятная логика, предположительно достанем многие из упоминаемых товаров.

**Недостатки**: сложная реализация, WordNet и/или векторная модель могут не помочь.

**3.** С помощью WordNet и/или векторной модели можно подобрать гипонимы, гиперонимы и синонимы к категории для каждого продукта. Затем эти синонимы искать срели отзывов.

**Дополнительные данные**: Metadata, WordNet и/или векторная модель

**Достоинства**: понятная логика, предположительно достанем многие из упоминаемых товаров, не очень сложная реализация.

**Недостатки**: не понятно, насколько качественные гиперонимы мы получим, и насколько гипотеза об употреблении гиперонимо/гипонимов/синонимов вместо непосредственно названий товаров верна.

### Задание 2.
Реализуйте один из предложенных вами способов.

Я попробую использовать комбинацию 2 и 3 способов.

#### Загрузка данных

In [ ]:
! wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Pet_Supplies.json.gz

--2021-12-04 11:25:24--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Pet_Supplies.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284871978 (272M) [application/octet-stream]
Saving to: ‘meta_Pet_Supplies.json.gz.1’

meta_Pet_Supplies.j 100%[===================>] 271.67M  20.8MB/s    in 14s     

2021-12-04 11:25:38 (19.6 MB/s) - ‘meta_Pet_Supplies.json.gz.1’ saved [284871978/284871978]



In [ ]:
df_meta = getDF('meta_Pet_Supplies.json.gz')
titles = df_meta['title']
titles = titles.tolist()
len(titles)

205999

In [ ]:
titles[:20]

['DR.REXY Hemp Oil for Dogs and Cats - 100% Organic Food Supplement - Full Spectrum Hemp Extract - Pain Relief for Dogs &amp; Cats, Calming, Stress &amp; Anxiety Support, Wellness, Hip &amp; Joint Health - 250mg',
 'Pet Qwerks Treat Cookbook with Cutters',
 'The Best of 101 Dog Tricks',
 'Lionfishes and Other Scorpionfishes: The Complete Guide to the Successful Care and Breeding of These Spectacular and Popular Marine Fish (Aquarium Success)',
 'Pet Media Feathered Phonics The Easy Way To Teach Your Bird To Speak Volume 1: 96 Words and Phrases',
 'Theory of Five Book by Melissa Verplank',
 "ABC's of Behavior Shaping; Proactive Behavior Management Seminar DVD Set",
 'Giant Clams in the Sea and the Aquarium',
 'Simply Charming Circles',
 'Piece of Pie A La Mode',
 'Pill Wrap Paste - 56 servings',
 'Pathway Dog Journal, Capture Your Memories, Pet Health and More',
 'Pathway Cat Journal, Capture Your Memories, Pet Health and More',
 'Lees Wooden Air Diffuser 2" Limewood 2/pk',
 'Cat Sitter

In [ ]:
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Pet Supplies, Dogs, Health Supplies, Hip &amp...",,[Dr. Rexy hemp oil has powerful anti-inflammat...,,DR.REXY Hemp Oil for Dogs and Cats - 100% Orga...,[],,DR.REXY,[Made strictly with organic derived ingredient...,"[>#93,463 in Grocery &amp; Gourmet Food (See T...",[],Amazon Home,,,$19.90,061539972X,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Pet Supplies, Dogs]",,[Know exactly what your dog is eating with 101...,,Pet Qwerks Treat Cookbook with Cutters,[],,Pet Qwerks,"[Recipe book, Cookie cutters, Indoor/outdoor use]","190,234 in Pet Supplies (","[B075DYQ1PH, 1604334657, 1604336919, 163026043...",Pet Supplies,,,$7.86,0615553605,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Pet Supplies, Dogs, Food, Dry]",,"[, , , ]",,The Best of 101 Dog Tricks,"[0760339740, 1592533256, 1592535305, 159253730...",,,[],"76,847 in Movies &amp; TV (","[1592533256, 0760339740, 1592537308, 159253530...",Movies &amp; TV,,,,0760339597,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[, The venomous but beautiful scorpionfishes a...",,Lionfishes and Other Scorpionfishes: The Compl...,"[1911142186, 1514291983]",,Tfh/Nylabone,[Used Book in Good Condition],"435,039 in Pet Supplies (",[],Pet Supplies,,,$24.99,0793816793,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Pet Supplies, Top Selection from AmazonPets]",,[Volume 1: 96 Words &amp; Phrases! This is the...,,Pet Media Feathered Phonics The Easy Way To Te...,"[B0002FP328, B0002FP32S, B0002FP32I, B00CAMARX...",,Pet Media,[Award-winning audio CD teaches parrots how to...,"52,435 in Pet Supplies (","[B0002FP328, B00CAMARXG, B0002FP32S, B0013L6CA...",Pet Supplies,,,$6.97,0972585419,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


Составим необходимый датафрейм

In [ ]:
small_df = pd.merge(df, df_meta, on='asin')
small_df = small_df[['asin', 'reviewText', 'title', 'brand', 'category']]
small_df.head()

,asin,reviewText,title,brand,category
0,0972585419,I purchased this cd for my Pocket Parrot. It h...,Pet Media Feathered Phonics The Easy Way To Te...,Pet Media,"[Pet Supplies, Top Selection from AmazonPets]"
1,0972585419,"Maybe it's just my Amazon parrot, but she's no...",Pet Media Feathered Phonics The Easy Way To Te...,Pet Media,"[Pet Supplies, Top Selection from AmazonPets]"
2,0972585419,I bought this to help me teach my blue quaker ...,Pet Media Feathered Phonics The Easy Way To Te...,Pet Media,"[Pet Supplies, Top Selection from AmazonPets]"
3,0972585419,Bird showed no interest in it....,Pet Media Feathered Phonics The Easy Way To Te...,Pet Media,"[Pet Supplies, Top Selection from AmazonPets]"
4,0972585419,Didn't do a thing for my African Grey! He hate...,Pet Media Feathered Phonics The Easy Way To Te...,Pet Media,"[Pet Supplies, Top Selection from AmazonPets]"


In [ ]:
small_df[20:27]

,asin,reviewText,title,brand,category
20,0978619404,My impressions of Fatherree's book:\n\nPros --...,Giant Clams in the Sea and the Aquarium,James W. Fatherree,"[Pet Supplies, Fish & Aquatic Pets, Aquarium D..."
21,0978619404,finish reading this book in 2 days. this book ...,Giant Clams in the Sea and the Aquarium,James W. Fatherree,"[Pet Supplies, Fish & Aquatic Pets, Aquarium D..."
22,0978619404,There aren't enough books on Giant Clams for t...,Giant Clams in the Sea and the Aquarium,James W. Fatherree,"[Pet Supplies, Fish & Aquatic Pets, Aquarium D..."
23,0978619404,well written and great pics. A keeper for you...,Giant Clams in the Sea and the Aquarium,James W. Fatherree,"[Pet Supplies, Fish & Aquatic Pets, Aquarium D..."
24,0978619404,I enjoyed this book. It presents basic importa...,Giant Clams in the Sea and the Aquarium,James W. Fatherree,"[Pet Supplies, Fish & Aquatic Pets, Aquarium D..."
25,1223000893,I purchased the Trilogy with hoping my two cat...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,[]
26,1223000893,My cats don't watch TV... If they can't smell...,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",Go Cat / Sunrise,[]


In [ ]:
small_df.to_csv('small_df.csv')

#### Препроц

In [ ]:
def preproc(raw):
    clear_raw = []
    doc = nlp(raw.lower())
    for token in doc:
        if token.text.isalpha():
            clear_raw.append((token.lemma_, token.pos_))
    return clear_raw

#### Список всех товаров

Нашими NE будут:

- сущ
- сущ+сущ
- прил+сущ
- сущ+предлог+сущ

In [ ]:
products = []
preproc_titles = {}
for j, title in enumerate(set(small_df['title'])):
    if j % 10000 == 0:
        print(j)
    title = preproc(title)
    for (w,t) in title:
        if t=='NOUN':
            products.append(w)
    bigrams = nltk.bigrams(title)
    trigrams = nltk.trigrams(title)
    try:
        for (w1,t1), (w2,t2) in bigrams:
            if (t1=='NOUN') and (t2=='NOUN') or (t1=='ADJ') and (t2=='NOUN'):
                products.append(' '.join([w1, w2]))
    except RuntimeError as error:
        pass
    try:
        for (w1,t1), (w2,t2), (w3,t3) in trigrams:
            if (t1=='NOUN') and (t2=='ADP') and (t3=='NOUN'):
                products.append(' '.join([w1, w2, w3]))
    except RuntimeError as error:
        pass

0
10000
20000
30000
40000


In [ ]:
products = set(products)
products = list(products)
print(products[0], '\n', products[128], '\n', products[32000])

superfood for dog 
 sense cat 
 collar for pound


#### Товары в отзывах

Теперь препроц для ревью

In [ ]:
def preproc_review(raw):
    raw = str(raw).lower().strip()
    tokens = word_tokenize(raw)
    clean_text = []
    for tok in tokens:
        if tok.isalpha():
            clean_text.append(lemmatizer.lemmatize(tok))
    return clean_text

Ищем пересечения

In [ ]:
ne = []
for j, review in enumerate(set(small_df['reviewText'])):
    if j % 25000 == 0:
        print(j)
    review = preproc_review(str(review))
    bigrams = ngrams(review, 2)
    bi = []
    try:
        for w1, w2 in bigrams:
            bi.append(' '.join([w1, w2]))
    except RuntimeError as error:
        pass
    trigrams = ngrams(review, 3)
    tri = []
    try:
        for w1, w2, w3 in trigrams:
            tri.append(' '.join([w1, w2, w3]))
    except RuntimeError as error:
        pass
    ne.extend(list(set(products)&set(review)))
    ne.extend(list(set(products)&set(bi)))
    ne.extend(list(set(products)&set(tri)))
print(len(ne))

0
25000
50000
75000
100000
125000
150000
175000
200000
225000
250000
275000
300000
325000
350000


KeyboardInterrupt: ignored

In [ ]:
print(len(ne))

5245952


In [ ]:
print(len(small_df))

2336465


In [ ]:
import json 

with open("ne_list.json", "w", encoding="utf-8") as file:
    json.dump(ne, file)

1) я поставила программу работать и забыла про неё, не увидев, что все 3 часа моего отсутсвия она работала. Предполагаю, что проблема в препроцессинге.

2) перед тем, как искать пересечения товаров из названий с отзывами, я забыла расширить список товаров с помощью векторной модели и/или WordNet. Ниже я приведу другой способ решения, в котором использую векторную модель.

3) В датафреме 2 миллиона строк. Из них обработалось 350 тысяч за 3 часа. В списке NE получилось 5,2 миллиона элементов. Я хочу посмотреть, что там получилось, поэтому не буду удалять эту часть решения, хоть она и не соответствует выбранным способам.

In [ ]:
with open('ne_list.json', 'r', encoding='utf-8') as file:
    ne_list = json.load(file)

In [ ]:
ne_c = Counter(ne_list)
print(ne_c.most_common(25))

[('a', 200145), ('dog', 121074), ('in', 115922), ('on', 95049), ('love', 93592), ('one', 67324), ('cat', 63920), ('get', 52015), ('up', 51604), ('out', 50701), ('ha', 50398), ('work', 47500), ('good', 47424), ('well', 47386), ('product', 46695), ('can', 43551), ('use', 40625), ('time', 39094), ('more', 38182), ('little', 35054), ('no', 34944), ('other', 33765), ('about', 32437), ('food', 31418), ('small', 28077)]


Окей, с существительными не получилось. Давайте посмотрим на триграммы.

In [ ]:
ne_c_bi = Counter([i for i in ne_list if len(i.split())>1])
for ne in ne_c_bi.most_common(25):
    print(ne)

('dog love', 17891)
('a long', 6941)
('dog food', 5343)
('litter box', 4704)
('small dog', 4363)
('good quality', 4137)
('pet store', 3445)
('cat food', 3314)
('high quality', 2561)
('perfect size', 2540)
('dry food', 2416)
('german shepherd', 2388)
('other dog', 2376)
('large dog', 2138)
('good price', 2071)
('other brand', 1988)
('great quality', 1938)
('favorite toy', 1881)
('little dog', 1845)
('few month', 1716)
('wet food', 1674)
('cat litter', 1623)
('big dog', 1521)
('dog can', 1520)
('in case', 1498)


Окей, не супер информативно, тут есть какие-то товары (*dry food*, *favourite toy* (скорее к следующему заданию), *wet food*, *litter box*, *dog food*, *cat food*, *dog food*, *cat litter*), но они находятся среди кучи мусора. Посмотрим на триграммы. 

In [ ]:
ne_c_tri = Counter([i for i in ne_list if len(i.split())>2])
for ne in ne_c_tri.most_common(25):
    print(ne)

('tug of war', 561)
('lot of fun', 315)
('peace of mind', 305)
('cup of food', 174)
('hour of fun', 158)
('side by side', 152)
('ease of use', 138)
('gallon of water', 127)
('type of dog', 125)
('dog at night', 112)
('quality of life', 103)
('treat for dog', 102)
('toy for dog', 95)
('dog with allergy', 91)
('treat for training', 91)
('product for dog', 88)
('kind of dog', 84)
('bag of cat', 82)
('food for dog', 71)
('type of cat', 69)
('food for cat', 66)
('breed of dog', 63)
('piece for training', 55)
('toy for cat', 55)
('inch in diameter', 54)


Ну тут тоже не особенно информативно: *tug of war* (я не знала, но, оказывается, это вот такая [игрушка](https://www.thesprucepets.com/thmb/TRg0re5nWs_sxzZg57ASN6T9i8A=/1468x979/filters:fill(auto,1)/tug-of-war-with-your-dog-1118370_1287-c9bb58ede5234abbbacd6576f8db2f3c.jpg) для собак!), *cup of food*, *treat for dog*, *toy for dog*, *treat for training*, *product for dog*, *bag of cat*, *food for cat*, *toy for cat*). А что если попробовать из уже выделенных сущностей вытащить только те, которые имеют отношение к домашним животным?

#### Попробуем через w2v модель

In [ ]:
def process_w2v(raw):
    raw = str(raw).lower().strip()
    for char in punctuation:
        raw = raw.replace(char, ' ')
    return raw.split()

In [ ]:
corp = [process_w2v(i) for i in small_df[:500000].reviewText.tolist()]

In [ ]:
model = gensim.models.Word2Vec(corp, window=3, sg=0)

In [ ]:
pets_ne = []
pet_words = ['food', 'toy', 'bag', 'medicine']
for word in pet_words:
    model_words = []
    for (w, score) in model.wv.most_similar(word, topn=15):
        model_words.append(w)
    for model_word in model_words:
        for ne in ne_list:
            if model_word in ne.split():
                pets_ne.append(ne)

In [ ]:
pets_ne_c = Counter(pets_ne)
for ne in pets_ne_c.most_common(25):
    print(ne)

('product', 46695)
('thing', 20270)
('box', 12546)
('ball', 9366)
('stuff', 9064)
('bed', 8738)
('item', 7644)
('litter box', 4704)
('bottle', 4310)
('bone', 4189)
('package', 3365)
('diet', 3312)
('container', 2880)
('formula', 1646)
('tray', 1466)
('seed', 1191)
('pan', 1127)
('treatment', 1120)
('quality product', 1084)
('tennis ball', 1027)
('dog bed', 994)
('duck', 976)
('hay', 943)
('wellness', 926)
('game', 883)


Окей, кажется, так получше. Давайте попробуй посомтреть отдельно на биграммы, триграммы и униграммы.

In [ ]:
data = {1:[], 2:[], 3:[]}
for i in range(1, 4, 1):
    c = Counter([n for n in pets_ne if len(n.split())==i])
    for (w, score) in c.most_common(25):
        data[i].append(w)
popular_ne = pd.DataFrame(data, columns=data.keys())
popular_ne

,1,2,3
0,product,litter box,product for dog
1,thing,quality product,bed on top
2,box,tennis ball,product for cat
3,ball,dog bed,product for pet
4,stuff,water bottle,probiotic for dog
5,bed,science diet,ball for dog
6,item,litter pan,ball with bell
7,bottle,pet bed,box for cat
8,bone,pet product,pull out tray
9,package,treat ball,container for dog


Кстати, даже отсюда видно, что самые популярные названия для товаров - *product*, *thing*, *item*, *stuff*, а не непосредственно упоминания. И самые популярные домашние животные - кошки и собаки)

Вот так, кажется, стало хорошо. Для следующего задания выберем следующие 9 продуктов: извините, что не 5, интересно посмотреть на многие)

In [35]:
supplies = {1 : ['box', 'bone', 'bed'],
            2 : ['food container', 'cat bed', 'litter box'],
            3 : ['box for cat', 'ball for dog', 'product for aquarium']}

### Задание 3.
Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [3]:
from tqdm.auto import tqdm
import tqdm
tqdm.notebook.tqdm().pandas()

0it [00:00, ?it/s]

In [8]:
smaller_df = pd.read_csv('small_df.csv')

In [9]:
smaller_df = smaller_df[:350000]

In [10]:
def preproc_df_review(raw):
    raw = str(raw).lower().strip()
    tokens = word_tokenize(raw)
    clean_text = ''
    for tok in tokens:
        if tok.isalpha():
            clean_text = clean_text + ' ' + lemmatizer.lemmatize(tok)
    return clean_text

In [11]:
smaller_df['clean_review'] = smaller_df['reviewText'].progress_apply(lambda x: preproc_df_review(x))

  0%|          | 0/350000 [00:00<?, ?it/s]

In [ ]:
smaller_df.to_csv('smaller_df.csv')

Будем собирать только биграммы.

In [6]:
smaller_df = pd.read_csv('smaller_df.csv')

Биграммы для униграмм

In [36]:
bigrams = ngrams(' '.join(smaller_df['clean_review']).split(), 2)
bi = Counter(bigrams)
supply_bigrams = {supply : [] for supply in supplies[1]}
for bigram in bi.most_common():
    for supply in supply_bigrams.keys():
        if supply in bigram[0]:
            supply_bigrams[supply].append(bigram)

Биграммы для биграмм (сосед + биграмма или биграмма + сосед)

In [13]:
trigrams = ngrams(' '.join(smaller_df['clean_review']).split(), 3)
tri = Counter(trigrams)
supply_trigrams = {supply : [] for supply in supplies[2]}
for trigram in tri.most_common():
    for supply in supply_trigrams.keys():
        if supply in ' '.join(trigram[0]):
            supply_trigrams[supply].append(trigram)

Биграммы для триграмм (сосед + триграмма или триграмма + сосед)

In [19]:
fourgrams = ngrams(' '.join(smaller_df['clean_review']).split(), 4)
four = Counter(fourgrams)
supply_fourgrams = {supply : [] for supply in supplies[3]}
for fourgram in four.most_common():
    for supply in supply_fourgrams.keys():
        if supply in ' '.join(fourgram[0]):
            supply_fourgrams[supply].append(fourgram)

Для биграмм и триграмм получается мало элементов:

In [21]:
for k, v in supply_trigrams.items():
    print(len(v))

52
78
821


In [22]:
for k, v in supply_fourgrams.items():
    print(len(v))

5
21
23


In [37]:
df_bigrams = pd.DataFrame({i : [' '.join(j[0]) for j in supply_bigrams[i][:150]] for i in supply_bigrams.keys()})
df_bigrams

,box,bone,bed
0,litter box,this bone,the bed
1,the box,the bone,this bed
2,box and,these bone,bed and
3,box i,bone and,bed for
4,box is,bone i,dog bed
...,...,...,...
145,that box,bone though,bed soft
146,box fit,chewy bone,bed then
147,music box,souper bone,bed more
148,box our,softer bone,gravel bed


In [26]:
df_trigrams = pd.DataFrame({i : [' '.join(j[0]) for j in supply_trigrams[i][:52]] for i in supply_trigrams.keys()})
df_trigrams

,food container,cat bed,litter box
0,the food container,a cat bed,the litter box
1,dog food container,cat bed i,litter box and
2,cat food container,the cat bed,this litter box
3,fish food container,this cat bed,litter box i
4,food container and,cat bed and,a litter box
5,food container in,nice cat bed,cat litter box
6,pet food container,cat bed is,litter box it
7,food container i,cat bed for,litter box for
8,food container can,cat bed it,litter box is
9,a food container,other cat bed,litter box the


In [27]:
df_fourgrams = pd.DataFrame({i : [' '.join(j[0]) for j in supply_fourgrams[i][:5]] for i in supply_fourgrams.keys()})
df_fourgrams

,box for cat,ball for dog,product for aquarium
0,box for cat and,ball for dog that,good product for aquarium
1,the box for cat,tennis ball for dog,best product for aquarium
2,litter box for cat,ball for dog you,great product for aquarium
3,open box for cat,jolly ball for dog,product for aquarium maintenance
4,box for cat to,the ball for dog,product for aquarium water


С триграммами как-то не очень получилось, их рассматривать дальше не будем.

### Задание 4.
Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

#### Униграммы

In [30]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(' '.join(smaller_df['clean_review']).split())
finder.apply_freq_filter(10)

In [63]:
for supply in supply_bigrams.keys():
    lem = lemmatizer.lemmatize(supply)
    rate = [i for i in finder.nbest(bigram_measures.pmi, 100000000000) if lem in i][:150]
    df_bigrams[f'{lem}_pmi'] = [' '.join(i) for i in rate]

In [64]:
for supply in supply_bigrams.keys():
    lem = lemmatizer.lemmatize(supply)
    rate = [i for i in finder.nbest(bigram_measures.student_t, 100000000000) if lem in i][:150]
    df_bigrams[f'{lem}_tscore'] = [' '.join(i) for i in rate]

In [62]:
for supply in supply_bigrams.keys():
    lem = lemmatizer.lemmatize(supply)
    rate = [i for i in finder.nbest(bigram_measures.dice, 100000000000) if lem in i][:150]
    df_bigrams[f'{lem}_dice'] = [' '.join(i) for i in rate]

In [72]:
df_bigrams

,box,bone,bed,box_pmi,bone_pmi,bed_pmi,box_tscore,bone_tscore,bed_tscore,box_dice,bone_dice,bed_dice
0,litter box,this bone,the bed,voice box,cuttle bone,kuranda bed,litter box,this bone,this bed,litter box,edible bone,pet bed
1,the box,the bone,this bed,littler box,nyla bone,orthopedic bed,the box,these bone,the bed,big box,these bone,fleece bed
2,box and,these bone,bed and,litter box,marrow bone,flower bed,box and,the bone,pet bed,the box,milk bone,his bed
3,box i,bone and,bed for,nesting box,goodie bone,bolster bed,box i,chew bone,dog bed,cardboard box,chew bone,bed fit
4,box is,bone i,dog bed,uncovered box,bone repairing,cuddly bed,big box,bone and,bed for,box store,nyla bone,crate bed
...,...,...,...,...,...,...,...,...,...,...,...,...
145,that box,bone though,bed soft,filter box,different bone,bed crate,flat box,same bone,bed into,box filter,bone seems,bed it
146,box fit,chewy bone,bed then,box also,smaller bone,small bed,original box,bone left,bed once,receptacle box,same bone,best bed
147,music box,souper bone,bed more,box say,good bone,bed were,normal box,bone almost,small bed,breeding box,bone even,bed into
148,box our,softer bone,gravel bed,box there,bone seems,big bed,tiny box,bone seems,good bed,box fit,bone out,small bed


Выведем 10 лучших по каждому критерию

In [73]:
df_bigrams.iloc[:10,3:]

,box_pmi,bone_pmi,bed_pmi,box_tscore,bone_tscore,bed_tscore,box_dice,bone_dice,bed_dice
0,voice box,cuttle bone,kuranda bed,litter box,this bone,this bed,litter box,edible bone,pet bed
1,littler box,nyla bone,orthopedic bed,the box,these bone,the bed,big box,these bone,fleece bed
2,litter box,marrow bone,flower bed,box and,the bone,pet bed,the box,milk bone,his bed
3,nesting box,goodie bone,bolster bed,box i,chew bone,dog bed,cardboard box,chew bone,bed fit
4,uncovered box,bone repairing,cuddly bed,big box,bone and,bed for,box store,nyla bone,crate bed
5,liter box,galileo bone,fleece bed,box is,bone is,bed is,voice box,real bone,travel bed
6,hooded box,flexichew bone,comfy bed,cat box,bone for,bed and,cat box,bone shaped,this bed
7,music box,knuckle bone,travel bed,this box,edible bone,his bed,liter box,rawhide bone,before bed
8,nest box,durachew bone,cozy bed,box but,bone are,her bed,sound box,white bone,nice bed
9,rectangular box,chilly bone,bed sheet,box with,bone but,bed i,shipping box,flavored bone,her bed


#### Биграммы

In [87]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(' '.join(smaller_df['clean_review']).split())
finder.apply_freq_filter(5)

In [89]:
df_trigrams = df_trigrams[:8]

In [93]:
for supply in supply_trigrams.keys():
    rate = [i for i in finder.nbest(trigram_measures.pmi, 100000000000) if supply in ' '.join(i)][:8]
    df_trigrams[f'{supply}_pmi'] = [' '.join(i) for i in rate]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [97]:
for supply in supply_trigrams.keys():
    rate = [i for i in finder.nbest(trigram_measures.student_t, 100000000000) if supply in ' '.join(i)][:8]
    df_trigrams[f'{supply}_tscore'] = [' '.join(i) for i in rate]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Для триграмм в [nltk](https://tedboy.github.io/nlps/generated/generated/nltk.TrigramAssocMeasures.html#nltk.TrigramAssocMeasures) нет метода Dice, поэтому возьмём Jaccard.

In [98]:
for supply in supply_trigrams.keys():
    rate = [i for i in finder.nbest(trigram_measures.jaccard, 100000000000) if supply in ' '.join(i)][:8]
    df_trigrams[f'{supply}_jaccard'] = [' '.join(i) for i in rate]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [99]:
df_trigrams

,food container,cat bed,litter box,food container_pmi,cat bed_pmi,litter box_pmi,food container_tscore,cat bed_tscore,litter box_tscore,food container_jaccard,cat bed_jaccard,litter box_jaccard
0,the food container,a cat bed,the litter box,fish food container,nice cat bed,hooded litter box,the food container,a cat bed,the litter box,fish food container,nice cat bed,cat litter box
1,dog food container,cat bed i,litter box and,pet food container,small cat bed,litter box avoidance,dog food container,this cat bed,litter box and,cat food container,small cat bed,covered litter box
2,cat food container,the cat bed,this litter box,cat food container,other cat bed,automatic litter box,cat food container,cat bed i,this litter box,pet food container,other cat bed,dome litter box
3,fish food container,this cat bed,litter box i,dog food container,cat bed just,uncovered litter box,fish food container,nice cat bed,litter box i,dog food container,cat bed just,the litter box
4,food container and,cat bed and,a litter box,food container in,this cat bed,sifting litter box,food container and,cat bed is,a litter box,food container in,this cat bed,this litter box
5,food container in,nice cat bed,cat litter box,the food container,cat bed is,dome litter box,pet food container,the cat bed,cat litter box,the food container,a cat bed,regular litter box
6,pet food container,cat bed is,litter box it,food container and,a cat bed,covered litter box,food container in,small cat bed,litter box it,food container and,cat bed is,their litter box
7,food container i,cat bed for,litter box for,food container i,cat bed for,enclosed litter box,food container i,other cat bed,litter box for,food container i,cat bed i,automatic litter box


### Задание 5.
 Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.

#### Биграммы для униграмм

На первый взгляд PMI справляется лучше остальных. Вряд ли нам нужны биграммы NE + правый сосед. Давайте выведем побольше биграмм, отранжированных PMI, но только с левым соседом.

In [74]:
df_bigrams.iloc[:25,3:6]

,box_pmi,bone_pmi,bed_pmi
0,voice box,cuttle bone,kuranda bed
1,littler box,nyla bone,orthopedic bed
2,litter box,marrow bone,flower bed
3,nesting box,goodie bone,bolster bed
4,uncovered box,bone repairing,cuddly bed
5,liter box,galileo bone,fleece bed
6,hooded box,flexichew bone,comfy bed
7,music box,knuckle bone,travel bed
8,nest box,durachew bone,cozy bed
9,rectangular box,chilly bone,bed sheet


#### Биграммы для биграмм

Тут совсем мало записей, поэтому просто выведем все, которые выглядят как левый сосед + NE.

In [111]:
for supply in supplies[2]:
    supply_trigr = []
    print('------')
    print(supply)
    print('------')
    for column in df_trigrams.columns:
        if column.startswith(supply):
            for trigr in df_trigrams[column].tolist():
                if not trigr.startswith(supply):
                    supply_trigr.append(trigr)
    supply_trigr = set(supply_trigr)
    for s in supply_trigr:
        print(s)

------
food container
------
fish food container
cat food container
dog food container
pet food container
the food container
------
cat bed
------
the cat bed
small cat bed
a cat bed
this cat bed
other cat bed
nice cat bed
------
litter box
------
uncovered litter box
a litter box
the litter box
sifting litter box
enclosed litter box
dome litter box
automatic litter box
their litter box
this litter box
hooded litter box
regular litter box
cat litter box
covered litter box
